In [32]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS

%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()

from tqdm import tqdm

In [2]:
student_exp_name='samira_fd1'
teacher_exp_name='0.0001_offlineteacher_v3'
teacher_config='small_lstm_v4'
task_name = 'word_sv_agreement_vp'
student_model='cl_gpt2'
teacher_model='cl_lstm'
student_config='small_gpt_v9'
distill_config='pure_distill_2'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [15]:
student_exp_name='samira_fd0'
teacher_exp_name='0.0001_offlineteacher_v3'
teacher_config='small_lstm_v4'
task_name = 'word_sv_agreement_vp'
student_model='cl_lstm'
teacher_model='cl_lstm'
student_config='small_lstm_v4'
distill_config='pure_distill_2'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [33]:
student_exp_name='lisa_fd4'
teacher_exp_name='0.0001_offlineteacher_v1'
teacher_config='small_gpt_v9'
task_name = 'word_sv_agreement_vp'
student_model='cl_gpt2'
teacher_model='cl_gpt2'
student_config='small_gpt_v9'
distill_config='pure_distill_2'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [42]:
student_exp_name='lisa_fd12'
teacher_exp_name='0.0001_offlineteacher_v1'
teacher_config='small_gpt_v9'
task_name = 'word_sv_agreement_vp'
student_model='cl_lstm'
teacher_model='cl_gpt2'
student_config= 'small_lstm_v4'
distill_config='pure_distill_2'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [43]:
task = TASKS[task_name](get_task_params(), data_dir='../data')

Vocab len:  10034


In [44]:
cl_token = task.databuilder.sentence_encoder().encode(constants.bos)
teacher_model = MODELS[teacher_model](hparams=get_model_params(task, teacher_model, teacher_config), cl_token=cl_token)
std_hparams=get_model_params(task, student_model, student_config)
std_hparams.output_attentions = True
std_hparams.output_embeddings = True
student_model = MODELS[student_model](
std_hparams, cl_token=cl_token)

model config: small_gpt_v9
{'embedding_dim': 128, 'resid_pdrop': 0.4, 'embd_pdrop': 0.2, 'attn_pdrop': 0.6, 'initializer_range': 0.05}
model config: small_lstm_v4
{'hidden_dim': 256, 'embedding_dim': 256, 'depth': 2, 'hidden_dropout_rate': 0.8, 'input_dropout_rate': 0.2, 'initializer_range': 0.1}


In [45]:
student_ckpt_dir = os.path.join(chkpt_dir, task.name,
                              '_'.join([distill_mode,distill_config,
                                        "teacher", teacher_model.model_name, 
                                        teacher_config,
                                        teacher_exp_name,
                                       "student",student_model.model_name,
                                        str(student_config),
                                        student_exp_name]))
print("student_checkpoint:", student_ckpt_dir)

student_ckpt = tf.train.Checkpoint(net=student_model)
student_manager = tf.train.CheckpointManager(student_ckpt, student_ckpt_dir, max_to_keep=None)

student_ckpt.restore(student_manager.latest_checkpoint)
if student_manager.latest_checkpoint:
  print("Restored student from {}".format(student_manager.latest_checkpoint))

student_model.compile(loss=task.get_loss_fn(), metrics=task.metrics())

student_checkpoint: ../tf_ckpts/word_sv_agreement_vp/offline_pure_distill_2_teacher_cl_gpt2_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_0.0001_offlineteacher_v1_student_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_lisa_fd12


Restored student from ../tf_ckpts/word_sv_agreement_vp/offline_pure_distill_2_teacher_cl_gpt2_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_0.0001_offlineteacher_v1_student_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_lisa_fd12/ckpt-60


In [37]:
teacher_ckpt_dir = os.path.join(chkpt_dir, task.name,
                                  '_'.join([teacher_model.model_name, teacher_config,teacher_exp_name]))

teacher_ckpt = tf.train.Checkpoint(net=teacher_model)
teacher_manager = tf.train.CheckpointManager(teacher_ckpt, teacher_ckpt_dir, max_to_keep=None)

teacher_ckpt.restore(teacher_manager.latest_checkpoint)
if teacher_manager.latest_checkpoint:
  print("Restored teacher from {}".format(teacher_manager.latest_checkpoint))

Restored teacher from ../tf_ckpts/word_sv_agreement_vp/cl_gpt2_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_0.0001_offlineteacher_v1/ckpt-60


In [41]:
teacher_model.compile(loss=task.get_loss_fn(), metrics=task.metrics())
train = teacher_model.evaluate(task.train_dataset, steps=task.n_train_batches)
valid = teacher_model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
test = teacher_model.evaluate(task.test_dataset, steps=task.n_test_batches)

print("train:", train)
print("valid:", valid)
print("test:", test)

21933/21933 [==============================] - 347s 16ms/step - loss: 0.2137 - classification_loss: 0.1933 - sparse_categorical_accuracy: 0.9561
train: [0.030510716126663246, 0.010100857, 0.9956816]
valid: [0.20903526653149507, 0.18862544, 0.9559832]
test: [0.21371715798148913, 0.19330677, 0.9561206]


In [47]:
train = student_model.evaluate(task.train_dataset, steps=task.n_train_batches)
valid = student_model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
test = student_model.evaluate(task.test_dataset, steps=task.n_test_batches)

print("train:", train)
print("valid:", valid)
print("test:", test)

21933/21933 [==============================] - 498s 23ms/step - loss: 0.1517 - classification_loss: 0.1089 - sparse_categorical_accuracy: 0.9707
train: [0.1031347155622388, 0.060294516, 0.98126143]
valid: [0.1464964546172357, 0.10365635, 0.97237045]
test: [0.1517115405279227, 0.10887108, 0.9707347]


In [13]:
print("train:", train)
print("valid:", valid)
print("test:", test)

train: [0.10100768137928824, 0.05516132, 0.97829366]
valid: [0.12030364813782819, 0.07445732, 0.97224337]
test: [0.12261093113181942, 0.0767647, 0.9721524]


In [10]:
model_name='cl_gpt2'
model_config='small_gpt_v9'
learning_rate=0.0001
exp_name='offlineteacher_v1'

cl_token = task.databuilder.sentence_encoder().encode(constants.bos)
hparams=get_model_params(task, model_name, model_config)
hparams.output_attentions = True
hparams.output_embeddings = True

model = MODELS[model_name](hparams=hparams, cl_token=cl_token)


ckpt_dir = os.path.join(chkpt_dir,task.name,
                        model.model_name+"_"+str(model_config)+"_"+str(learning_rate)+"_"+exp_name)

ckpt = tf.train.Checkpoint(net=model)
manager = tf.train.CheckpointManager(ckpt, ckpt_dir, max_to_keep=None)

ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  print("Restored student from {}".format(manager.latest_checkpoint))

model.compile(loss=task.get_loss_fn(), metrics=task.metrics())
model.evaluate(task.train_dataset, steps=task.n_train_batches)
model.evaluate(task.valid_dataset, steps=task.n_valid_batches)
model.evaluate(task.train_dataset, steps=task.n_train_batches)

model config: small_gpt_v9
{'embedding_dim': 128, 'resid_pdrop': 0.4, 'embd_pdrop': 0.2, 'attn_pdrop': 0.6, 'initializer_range': 0.05}
Restored student from ../tf_ckpts/word_sv_agreement_vp/cl_gpt2_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_0.0001_offlineteacher_v1/ckpt-60
2464/2464 [==============================] - 42s 17ms/step - loss: 0.0305 - classification_loss: 0.0101 - sparse_categorical_accuracy: 0.9957


[0.030508206180575027, 0.010098332, 0.9956816]